In [ ]:
model_path = r'..\models\fl_fedavg\effnet-b0\ff_effnet0_1fc_fl.pth'
video_path = r'006_002.mp4'
#image_path = r'test_original.png'


import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
from efficientnet_pytorch import EfficientNet

from facenet_pytorch import MTCNN
from PIL import Image
import cv2
import numpy as np

device = 'cuda'
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=1)
# model = timm.create_model('xception', pretrained=True, num_classes=1)
model.load_state_dict(torch.load(model_path)['model_state'])
model.to(device)
model.eval() 

Loaded pretrained weights for efficientnet-b0


C:\Users\Sheryl\AppData\Local\Temp\ipykernel_30076\3779382812.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['model_state'

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [6]:
IMG_SIZE = 224


mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

class ImageTransform:
    def __init__(self, size, mean, std, train=False):
        self.data_transform = transforms.Compose([
            transforms.Resize((size, size), interpolation=Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

    def __call__(self, img):
        return self.data_transform(img)



transformer = ImageTransform(IMG_SIZE, mean, std, train=False)

In [7]:
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=1)
model.load_state_dict(torch.load(model_path)['model_state'])
model.to(device)
model.eval() 

Loaded pretrained weights for efficientnet-b0


C:\Users\Sheryl\AppData\Local\Temp\ipykernel_30076\1431730160.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['model_state']

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [8]:
mtcnn = MTCNN(margin=300, select_largest=False, factor=0.5, device=device, post_process=False) # post_process=False if want human readable image
cap = cv2.VideoCapture(video_path)

frame_interval = 32
frame_idx = 0
predictions = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_idx % frame_interval == 0:
        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(frame_rgb)

        boxes, probs = mtcnn.detect(frame_rgb, landmarks=False)

        if boxes is None:
            print(f"[Frame {frame_idx}] No face detected.")
        else:
            for i, box in enumerate(boxes):
                face = pil_img.crop((box[0], box[1], box[2], box[3]))
                face_tensor = transformer(face).unsqueeze(0).to(device)

                with torch.no_grad():
                    output = torch.sigmoid(model(face_tensor))
                    prob = output.item()
                    predicted_class = int(prob > 0.5)

                predictions.append({
                    'frame': frame_idx,
                    'face_idx': i,
                    'prob': prob,
                    'predicted_class': predicted_class
                })

                print(f"[Frame {frame_idx} - Face {i+1}] Class: {predicted_class}, Prob: {prob:.4f}")

    frame_idx += 1

cap.release()

[Frame 0 - Face 1] Class: 1, Prob: 0.9533
[Frame 32 - Face 1] Class: 1, Prob: 0.7189
[Frame 64 - Face 1] Class: 1, Prob: 0.8451
[Frame 96 - Face 1] Class: 1, Prob: 0.9131
[Frame 128 - Face 1] Class: 1, Prob: 0.6366
[Frame 160 - Face 1] Class: 1, Prob: 0.7244
[Frame 192 - Face 1] Class: 1, Prob: 0.7090
[Frame 224 - Face 1] Class: 1, Prob: 0.6655
[Frame 256 - Face 1] Class: 1, Prob: 0.8274
[Frame 288 - Face 1] Class: 1, Prob: 0.7385
